In [1]:
#Importamos las librerías
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('once')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Empezamos con los datos de ocasionales estandarizados y  codificados:

In [4]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/ADALAB/proyecto/project-da-promo-C-module-3-team-4-main/data/ocasionales_est_encod.csv',index_col=0)
df.head(3)

,registro,clima,temperatura,sens_termica,humedad,viento,ocasionales,fecha_nueva,festividad,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,ocasionales_BOX,año_map,estacion_map
0,1,2,-0.828948,-0.678635,1.295131,-0.376545,331,2018-01-01,New Year's Day,1,1,0,0,16.397820,0,0
1,2,2,-0.722312,-0.740034,0.507407,0.882110,131,2018-01-02,NaN,0,1,1,1,11.472437,0,0
2,3,1,-1.642047,-1.746977,-1.354487,0.861132,120,2018-01-03,NaN,0,1,2,1,11.076035,0,0


In [5]:
df.columns

Index(['registro', 'clima', 'temperatura', 'sens_termica', 'humedad', 'viento',
       'ocasionales', 'fecha_nueva', 'festividad', 'festividad_1', 'mes_bueno',
       'dia_semana_nuevo', 'no_laboral_nuevo', 'ocasionales_BOX', 'año_map',
       'estacion_map'],
      dtype='object')

In [6]:
df.drop(['registro','fecha_nueva','festividad', 'ocasionales_BOX'], axis=1,inplace=True)
df.columns

Index(['clima', 'temperatura', 'sens_termica', 'humedad', 'viento',
       'ocasionales', 'festividad_1', 'mes_bueno', 'dia_semana_nuevo',
       'no_laboral_nuevo', 'año_map', 'estacion_map'],
      dtype='object')

In [7]:
X=df.drop(['ocasionales'],axis=1)
y=df['ocasionales']

In [8]:
# Separamos en train y test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Creamos objeto del modelo
arbol=DecisionTreeRegressor(random_state=0)
#Ajustamos el modelo
arbol.fit(x_train,y_train)

DecisionTreeRegressor(random_state=0)

In [9]:
# Obtenemos la raíz cuadrada del número de variables predictoras de nuestro árbol
max_features = np.sqrt(len(x_train.columns))
max_features

3.3166247903554

In [10]:
# Obtenemos la profundidad del árbol representado
arbol.tree_.max_depth

20

In [11]:
# Predicción para los datos de entrenamiento
y_pred_train1 = arbol.predict(x_train)
# Predicción para los datos de test
y_pred_test1 = arbol.predict(x_test)

In [12]:
def obtener_metricas(y_test_m, y_train_m, y_predict_test_m, y_predict_train_m, modelo):
    # En un diccionario, calcular las métricas de train y test
    resultados_metricas = {
        'MAE': [mean_absolute_error(y_test_m, y_predict_test_m), mean_absolute_error(y_train_m, y_predict_train_m)],
        'MSE': [mean_squared_error(y_test_m, y_predict_test_m), mean_squared_error(y_train_m, y_predict_train_m)],
        'RMSE': [np.sqrt(mean_squared_error(y_test_m, y_predict_test_m)), np.sqrt(mean_squared_error(y_train_m, y_predict_train_m))],
        'R2':  [r2_score(y_test_m, y_predict_test_m), r2_score(y_train_m, y_predict_train_m)],
        "set": ["test", "train"], 
        "modelo": [modelo, modelo]}

    # Pasamos a dataframe el diccionario para su mejor visualizacion
    df_resultados_metricas = pd.DataFrame(resultados_metricas)

    return df_resultados_metricas

In [13]:
df_resultados1= obtener_metricas(y_test,y_train,y_pred_test1,y_pred_train1,'Decision Tree ocasionales Codif y Estand')
df_resultados1

,MAE,MSE,RMSE,R2,set,modelo
0,260.335616,166276.417808,407.770055,0.653897,test,Decision Tree ocasionales Codif y Estand
1,0.000000,0.000000,0.000000,1.000000,train,Decision Tree ocasionales Codif y Estand


In [14]:
# Definir diccionario con los hiperparámetros a modificar
# min_samples_split: número mínimo de observaciones que debe tener un nodo para dividirse, por defecto [10, 50, 100]
# min_samples_leaf: número mínimo de observaciones que debe tener cada uno de los nodos hijos para que se produzca la division, por defecto [10, 50, 100]
param = {"max_depth": [2,4,6], "max_features": [1,2,3], "min_samples_split": [10, 150, 300], "min_samples_leaf": [10,150,300]}

In [15]:
# Ajustar el modelo con GridSearchCV
# cv = 10 --> número de veces que hacemos la validación
# verbose = - 1 --> para no printear ningún mensaje en la pantalla
grid = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=param, cv=10, verbose=-1,
                        return_train_score=True, scoring="neg_mean_squared_error")

In [16]:
# Ajustar el modelo
grid.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 4, 6], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 150, 300],
                         'min_samples_split': [10, 150, 300]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [17]:
# El método nos dice cuál es el mejor modelo,
mejor_modelo = grid.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=6, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [18]:
y_pred_test_dt= mejor_modelo.predict(x_test)
y_pred_train_dt= mejor_modelo.predict(x_train)

In [19]:
df_resultados2= obtener_metricas(y_test,y_train,y_pred_test_dt,y_pred_train_dt,'Decision Tree ocasionales Codif y Estand II')
df_resultados2

,MAE,MSE,RMSE,R2,set,modelo
0,317.139869,230474.808632,480.077919,0.520268,test,Decision Tree ocasionales Codif y Estand II
1,304.668110,200968.639037,448.295259,0.570310,train,Decision Tree ocasionales Codif y Estand II


In [20]:
df.head(2)

,clima,temperatura,sens_termica,humedad,viento,ocasionales,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map
0,2,-0.828948,-0.678635,1.295131,-0.376545,331,1,1,0,0,0,0
1,2,-0.722312,-0.740034,0.507407,0.882110,131,0,1,1,1,0,0


Ahora con los datos sin estandarizar:

In [22]:
df2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ADALAB/proyecto/project-da-promo-C-module-3-team-4-main/data/ocasionales_encod.csv', index_col = 0)

In [23]:
df2.drop(['registro','fecha_nueva','festividad', 'ocasionales_BOX'], axis=1,inplace=True)
df2.columns

Index(['clima', 'temperatura', 'sens_termica', 'humedad', 'viento',
       'ocasionales', 'festividad_1', 'mes_bueno', 'dia_semana_nuevo',
       'no_laboral_nuevo', 'año_map', 'estacion_map'],
      dtype='object')

In [25]:
X=df.drop(['ocasionales'],axis=1)
y=df['ocasionales']

In [26]:
# Separamos en train y test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Creamos objeto del modelo
arbol=DecisionTreeRegressor(random_state=0)
#Ajustamos el modelo
arbol.fit(x_train,y_train)

DecisionTreeRegressor(random_state=0)

In [27]:
# Obtenemos la raíz cuadrada del número de variables predictoras de nuestro árbol
max_features = np.sqrt(len(x_train.columns))
max_features

3.3166247903554

In [28]:
# Obtenemos la profundidad del árbol representado
arbol.tree_.max_depth

20

In [29]:
# Predicción para los datos de entrenamiento
y_pred_train1 = arbol.predict(x_train)
# Predicción para los datos de test
y_pred_test1 = arbol.predict(x_test)

In [30]:
df_resultados3= obtener_metricas(y_test,y_train,y_pred_test1,y_pred_train1,'Decision Tree ocasionales Codif')
df_resultados3

,MAE,MSE,RMSE,R2,set,modelo
0,260.335616,166276.417808,407.770055,0.653897,test,Decision Tree ocasionales Codif
1,0.000000,0.000000,0.000000,1.000000,train,Decision Tree ocasionales Codif


In [31]:
# Definir diccionario con los hiperparámetros a modificar
# min_samples_split: número mínimo de observaciones que debe tener un nodo para dividirse, por defecto [10, 50, 100]
# min_samples_leaf: número mínimo de observaciones que debe tener cada uno de los nodos hijos para que se produzca la division, por defecto [10, 50, 100]
param = {"max_depth": [3,7,10], "max_features": [1,2,3], "min_samples_split": [10, 150, 300], "min_samples_leaf": [10,150,300]}

In [32]:
# Ajustar el modelo con GridSearchCV
# cv = 10 --> número de veces que hacemos la validación
# verbose = - 1 --> para no printear ningún mensaje en la pantalla
grid = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=param, cv=10, verbose=-1,
                        return_train_score=True, scoring="neg_mean_squared_error")

In [33]:
# Ajustar el modelo
grid.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [3, 7, 10], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 150, 300],
                         'min_samples_split': [10, 150, 300]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [34]:
# El método nos dice cuál es el mejor modelo,
mejor_modelo = grid.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=10, max_features=2, min_samples_leaf=10,
                      min_samples_split=10)

In [35]:
y_pred_test_dt= mejor_modelo.predict(x_test)
y_pred_train_dt= mejor_modelo.predict(x_train)

In [36]:
df_resultados4= obtener_metricas(y_test,y_train,y_pred_test_dt,y_pred_train_dt,'Decision Tree ocasionales Codif II')
df_resultados4

,MAE,MSE,RMSE,R2,set,modelo
0,364.300333,263183.490717,513.014123,0.452185,test,Decision Tree ocasionales Codif II
1,322.441306,231818.681431,481.475525,0.504350,train,Decision Tree ocasionales Codif II


In [37]:
df_resultados2

,MAE,MSE,RMSE,R2,set,modelo
0,317.139869,230474.808632,480.077919,0.520268,test,Decision Tree ocasionales Codif y Estand II
1,304.668110,200968.639037,448.295259,0.570310,train,Decision Tree ocasionales Codif y Estand II


In [40]:
df_metricas=pd.concat([df_resultados2,df_resultados4],axis=0)
df_metricas

,MAE,MSE,RMSE,R2,set,modelo
0,317.139869,230474.808632,480.077919,0.520268,test,Decision Tree ocasionales Codif y Estand II
1,304.668110,200968.639037,448.295259,0.570310,train,Decision Tree ocasionales Codif y Estand II
0,364.300333,263183.490717,513.014123,0.452185,test,Decision Tree ocasionales Codif II
1,322.441306,231818.681431,481.475525,0.504350,train,Decision Tree ocasionales Codif II


In [41]:
df_metricas.to_csv('/content/drive/My Drive/Colab Notebooks/ADALAB/proyecto/project-da-promo-C-module-3-team-4-main/data/metricas_ocas_dt.csv')